In [ ]:
!pip install -q findspark

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-12 00:34:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-11-12 00:34:35 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://finalprojects311042022.s3.us-west-2.amazonaws.com/Cleaned_Dataset.csv"
spark.sparkContext.addFile(url)
#df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("https://finalprojects311042022.s3.us-west-2.amazonaws.com/Cleaned_Dataset.csv"), sep="\t", header=True, inferSchema=True)
df = spark.read.csv(SparkFiles.get("Cleaned_Dataset.csv"), sep=",", header=True, inferSchema=True)
df.show()

+---+---+--------------+-------------+----+-------------+----------------------------------+----+---+----------------------------+----------------------------+---------------------------------+--------------------------------+----------------------------+------------------------------------+------------------------------+-----------------------------------+------------+-------------+-----------------+----------------+-------------+-------------+------------+-----------------------+----------+---------------+---------------+---------------+---------------+---------------+------+
|_c0|sex|coded_location|visit_outcome|freq|contact_count|time_of_visit_from_initial_contact|year|age|initial_contact_method_email|initial_contact_method_emrap|initial_contact_method_in_patient|initial_contact_method_in_person|initial_contact_method_other|initial_contact_method_self_referred|initial_contact_method_tabling|initial_contact_method_wom_referral|contact_None|contact_email|contact_inpatient|contact_inp

In [10]:
df.dtypes

[('_c0', 'int'),
 ('sex', 'double'),
 ('coded_location', 'double'),
 ('visit_outcome', 'string'),
 ('freq', 'int'),
 ('contact_count', 'int'),
 ('time_of_visit_from_initial_contact', 'int'),
 ('year', 'int'),
 ('age', 'int'),
 ('initial_contact_method_email', 'int'),
 ('initial_contact_method_emrap', 'int'),
 ('initial_contact_method_in_patient', 'int'),
 ('initial_contact_method_in_person', 'int'),
 ('initial_contact_method_other', 'int'),
 ('initial_contact_method_self_referred', 'int'),
 ('initial_contact_method_tabling', 'int'),
 ('initial_contact_method_wom_referral', 'int'),
 ('contact_None', 'int'),
 ('contact_email', 'int'),
 ('contact_inpatient', 'int'),
 ('contact_inperson', 'int'),
 ('contact_other', 'int'),
 ('contact_phone', 'int'),
 ('contact_text', 'int'),
 ('contact_unable_to_reach', 'int'),
 ('contact_vm', 'int'),
 ('visit_weekday_F', 'int'),
 ('visit_weekday_M', 'int'),
 ('visit_weekday_R', 'int'),
 ('visit_weekday_T', 'int'),
 ('visit_weekday_W', 'int'),
 ('bucket', 

In [28]:
df_success = df.filter(df.contact_count > 4)
df_success.show()

+---+---+--------------+-------------+----+-------------+----------------------------------+----+---+----------------------------+----------------------------+---------------------------------+--------------------------------+----------------------------+------------------------------------+------------------------------+-----------------------------------+------------+-------------+-----------------+----------------+-------------+-------------+------------+-----------------------+----------+---------------+---------------+---------------+---------------+---------------+------+
|_c0|sex|coded_location|visit_outcome|freq|contact_count|time_of_visit_from_initial_contact|year|age|initial_contact_method_email|initial_contact_method_emrap|initial_contact_method_in_patient|initial_contact_method_in_person|initial_contact_method_other|initial_contact_method_self_referred|initial_contact_method_tabling|initial_contact_method_wom_referral|contact_None|contact_email|contact_inpatient|contact_inp

In [31]:

mnew_df = df.filter(df.sex == 1.0)
mnew_df.show()


+---+---+--------------+-------------+----+-------------+----------------------------------+----+---+----------------------------+----------------------------+---------------------------------+--------------------------------+----------------------------+------------------------------------+------------------------------+-----------------------------------+------------+-------------+-----------------+----------------+-------------+-------------+------------+-----------------------+----------+---------------+---------------+---------------+---------------+---------------+------+
|_c0|sex|coded_location|visit_outcome|freq|contact_count|time_of_visit_from_initial_contact|year|age|initial_contact_method_email|initial_contact_method_emrap|initial_contact_method_in_patient|initial_contact_method_in_person|initial_contact_method_other|initial_contact_method_self_referred|initial_contact_method_tabling|initial_contact_method_wom_referral|contact_None|contact_email|contact_inpatient|contact_inp

In [38]:
M_count = new_df.count()
M_count

3107

In [36]:
fnew_df = df.filter(df.sex == 2.0)
fnew_df.show()

+---+---+--------------+-------------+----+-------------+----------------------------------+----+---+----------------------------+----------------------------+---------------------------------+--------------------------------+----------------------------+------------------------------------+------------------------------+-----------------------------------+------------+-------------+-----------------+----------------+-------------+-------------+------------+-----------------------+----------+---------------+---------------+---------------+---------------+---------------+------+
|_c0|sex|coded_location|visit_outcome|freq|contact_count|time_of_visit_from_initial_contact|year|age|initial_contact_method_email|initial_contact_method_emrap|initial_contact_method_in_patient|initial_contact_method_in_person|initial_contact_method_other|initial_contact_method_self_referred|initial_contact_method_tabling|initial_contact_method_wom_referral|contact_None|contact_email|contact_inpatient|contact_inp

In [37]:
F_count = fnew_df.count()
F_count

5485

In [40]:
total_count = df.count()
total_count

9150

In [42]:
male_percentage =  M_count / total_count * 100
male_percentage

33.95628415300547

In [43]:
female_percentage = F_count / total_count * 100
female_percentage

59.94535519125683